# Proof of burn

In [ ]:
%pip install ecdsa

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent.parent))

from elliptic_curves.instantiations.mnt4_753.mnt4_753 import MNT4_753
from tx_engine import Script, Tx, TxOut, Wallet

from token_examples.proof_of_burn.utils import generate_pob_utxo, load_and_process_vk, spend_proof_of_burn
from token_examples.tx_engine_utils import (
    p2pkh,
    setup_network_connection,
    spend_p2pkh,
    tx_from_id,
    tx_to_input,
    update_tx_balance,
)

ScalarFieldMNT4 = MNT4_753.scalar_field

## Set up the proving/verifying keys

The first thing you need to do is to set up a proving/verifying key pair for `burn_proof_system`. Head to `./tcp_proof_system` and execute the following command
```bash
cargo run --release -- --file ./configs/config_base/setup.toml --setup
```

Then, when the execution is complete, head to `./burn_proof_system` and execute the following command
```bash
cargo run --release -- setup
```

## Set up the public keys

If you already have a couple of keys, you can plug them in the cells below (in hex format). Otherwise, you can generate them with the following command:

```python
Wallet.generate_keypair("BSV_Testnet").to_hex()
```

In [ ]:
# Connection to the blockchain
connection = setup_network_connection("testnet")

In [ ]:
# The public key of the token holder
token_pub_key = Wallet.from_hexstr("BSV_Testnet", "")
token_pub_key.get_address()

'mvHjYDkkiuHfthPpu5TJ5GBaGFD96e3zMJ'

In [ ]:
# The public key of the token holder
issuer_pub_key = Wallet.from_hexstr("BSV_Testnet", "")
issuer_pub_key.get_address()

'mzNS1UFvupWoq5zffQZ5AFPznSbCc28tuZ'

## Get funding

Head over to your favourite faucet, e.g., [sCrypt Faucet](https://scrypt.io/faucet), and get fundings for the two public keys set up above. Paste the transaction ids of the funding transactions in the cell below.

In [ ]:
genesis_tx = tx_from_id("", connection)
issuer_funds = tx_from_id("", connection)

## Generate proof-of-burn UTXO

We now generate the UTXO that will prove a children of `genesis_tx` has been burnt. You don't need to to anything at this point, simply execute the following cells.

In [33]:
genesis_txid_as_input = ScalarFieldMNT4(int.from_bytes(genesis_tx.hash(), "little"))
vk, cache_vk, prepared_vk = load_and_process_vk(genesis_txid=genesis_txid_as_input.to_int())

In [34]:
proof_of_burn_utxo, change_utxo = generate_pob_utxo(
    vk=vk, prepared_vk=prepared_vk, issuer_funds=issuer_funds, issuer_pub_key=issuer_pub_key
)

In [35]:
proof_of_burn_tx, response = spend_p2pkh(
    txs=[issuer_funds],
    indices=[0],
    outputs=[proof_of_burn_utxo, change_utxo],
    index_output=1,
    public_keys=[issuer_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Failed to create proof of burn tx, error is {response.content}"

In [36]:
proof_of_burn_tx.id()

'12d1a59398cee68a10238e887c73b5c7579f0202ea2f866206d095864e60f5e7'

## Move the token two times

We now move the token a couple of times. To move the token, it is enough that you execute the following cells. However, to be able to generate the proof required to spend the proof-of-burn UTXO, you will need to generate a proof every time you move the token.

First of all, head to `./tcp_proof_system` and paste `genesis_txid` in `configs/config_base/prove.toml` both as `genesis_txid` and `outpoint_txid`. Then, execute the following command (from the root of `tcp_proof_system`):
```bash
cargo run --release -- --file ./configs/config_base/prove.toml --prove
```

Then, you execute the following cell to transfer the token for the first time.

In [37]:
token_tx_1, response = spend_p2pkh(
    txs=[genesis_tx],
    indices=[0],
    outputs=[p2pkh(token_pub_key, genesis_tx.tx_outs[0].amount)],
    index_output=0,
    public_keys=[token_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Token transfer failed, error is: {response.content}"

In [38]:
token_tx_1.serialize().hex()

'01000000017ed3c58428664807f6addc1992f6f50d741f92fa7efd092bf63172a931b11f05000000006b483045022100ac6b7954ec9791af4e1fd75508c76f6f792045350b1656b7ff22a374fda2721b02204ff73c3fa211afb6074fd8e6890c658672f47f56f8d29023d86c1f74fd8f3b284121034956aa2c326c774e85be9338941b0e9adc7815724126a918b01eae57d3715fc700000000013e860100000000001976a914a209d4cf940036062f833f96674a66cddce8a48288ac00000000'

In [39]:
token_tx_1.id()

'480679872c0f7c0d1144f21216c58407120bad0cfbd65665b9e2abfec0111378'

Now we generate the proof of the above transfer. Head to `./tcp_prof_system` and paste:
- `genesis_txid` in the respective field of `configs/config_rec_1/prove.toml`
- the txid of `token_tx_1` in the field `outpoint_txid` of the same file
- the serialisation of `token_tx_1` in the field `tx` of the same file

After having done so, execute the following command (from the root of `tcp_proof_system`):
```bash
cargo run --release -- --file ./configs/config_rec_1/prove.toml --prove
```

In [40]:
token_tx_2, response = spend_p2pkh(
    txs=[token_tx_1],
    indices=[0],
    outputs=[p2pkh(token_pub_key, token_tx_1.tx_outs[0].amount)],
    index_output=0,
    public_keys=[token_pub_key],
    fee_rate=10,
    network=connection,
)

assert response.status_code == 200, f"Token transfer failed, error is: {response.content}"

In [41]:
token_tx_2.serialize().hex()

'0100000001781311c0feabe2b96556d6fb0cad0b120784c51612f244110d7c0f2c87790648000000006b483045022100deb2dbc247883f7f0bf39c98e36d22ac4039741568bf8ecafc7137152a9ed82a02201c027f30afca9d3b96814b42d3b3f82ebce43622c48f25f1bec5aeac52f4882d4121034956aa2c326c774e85be9338941b0e9adc7815724126a918b01eae57d3715fc700000000013c860100000000001976a914a209d4cf940036062f833f96674a66cddce8a48288ac00000000'

In [42]:
token_tx_2.id()

'45ad468cfaf3f7d3d2b8efbabf7d71f0a6a1611686215351aded41029594fd40'

Once again, we need to generate the proof of the above transfer. Head to `./tcp_prof_system` and paste:
- `genesis_txid` in the respective field of `configs/config_rec_2/prove.toml`
- the txid of `token_tx_2` in the field `outpoint_txid` of the same file
- the serialisation of `token_tx_2` in the field `tx` of the same file

After having done so, execute the following command (from the root of `tcp_proof_system`):
```bash
cargo run --release -- --file ./configs/config_rec_2/prove.toml --prove
```

## Burning transaction

We are now going to burn the child of `genesis_tx` held in `token_tx_2`. First of all, we prepare the structure of the burning transaction (3 inputs, 2 ouputs). No unlocking scripts are required at this point.

In [43]:
inputs = [
    tx_to_input(token_tx_2, 0, Script()),
    tx_to_input(proof_of_burn_tx, 0, Script()),
    tx_to_input(proof_of_burn_tx, 1, Script()),
]

outputs = [
    TxOut(amount=0, script_pubkey=Script.parse_string("OP_0 OP_RETURN")),
    p2pkh(issuer_pub_key, amount=proof_of_burn_tx.tx_outs[1].amount + token_tx_2.tx_outs[0].amount),
]

spending_tx = Tx(
    version=1,
    tx_ins=inputs,
    tx_outs=outputs,
    locktime=0,
)

# Fee rate chosen to get 10 sat/KB after adding the unlocking script
spending_tx = update_tx_balance(spending_tx, 1, (178 + 300000) // 178 * 10)

In [44]:
spending_tx.serialize().hex()

'010000000340fd94950241edad515321861661a1a6f0717dbfbaefb8d2d3f7f3fa8c46ad45000000000000000000e7f5604e8695d00662862fea02029f57c7b5737c888e23108ae6ce9893a5d112000000000000000000e7f5604e8695d00662862fea02029f57c7b5737c888e23108ae6ce9893a5d11201000000000000000002000000000000000002006aecef0200000000001976a914cecdc7e19ac9b16e6d9a1a5c5195a97914dc430e88ac00000000'

Now, head to `./burn_proof_system` and paste:
- `genesis_txid` in the respective field of `data/proving_data.toml`
- the serialisation of `spending_tx` in the respective field of the same file

Then, execute the following command (from the root of `./burn_proof_system`)
```bash
cargo run --release -- prove
```

When the execution is completed, you can execute the following cell to publish `spending_tx` and burn the child of `genesis_txid`.

In [45]:
# Spend Proof-of-Burn UTXO
spending_tx, response = spend_proof_of_burn(
    outputs=spending_tx.tx_outs,
    cache_vk=cache_vk,
    token_tx=token_tx_2,
    pob_tx=proof_of_burn_tx,
    token_pub_key=token_pub_key,
    pob_pub_key=issuer_pub_key,
    network=connection,
)

assert response.status_code == 200, f"Spending of PoB failed, error is: {response.content}"

In [46]:
spending_tx.id()

'461fefe5f5b70746e5c477db3c57dbb6050c7e4842d3120e88c1077b19e1bdbc'